In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn

In [2]:
#Wczytanie i obróbka całości danych

big_dataset = pd.DataFrame()

#Trzeba wczytać każdy z 13 arkuszy z danymi dla poszczególnych okręgów wyborczych
for i in range(1,14):
    filename = '2019-european-parliament-election-in-poland-data\wyniki_gl_na_kand_po_obwodach_sheet_'+str(i)+'.csv'
    data_district = pd.read_csv(filename)
    data_district['obwod'] = i
    df = data_district.copy()
    cols_to_remove = []
    for c in range(35,len(df.columns)):
        cols_to_remove.append(c)
    #Wycięcie nadmiarowych danych - szczegółowych informacji o głosach na poszczególnych kandydatów w poszczególnych komisjach
    df.drop(df.columns[cols_to_remove],axis=1,inplace=True)
    #Uwzględnienie w danych zbiorczych wyników komitetów wyborczych w poszczególnych komisjach
    df['Konfederacja'] = data_district['Lista nr 1 - KWW KONFEDERACJA KORWIN BRAUN LIROY NARODOWCY']
    df['Wiosna'] = data_district['Lista nr 2 - KW WIOSNA ROBERTA BIEDRONIA']
    df['Koalicja Europejska'] = data_district['Lista nr 3 - KKW KOALICJA EUROPEJSKA PO PSL SLD .N ZIELONI']
    df['Prawo i Sprawiedliwość'] = data_district['Lista nr 4 - KW PRAWO I SPRAWIEDLIWOŚĆ']
    df['Lewica Razem'] = data_district['Lista nr 5 - KKW LEWICA RAZEM - RAZEM, UNIA PRACY, RSS']
    df['Kukiz15'] = data_district['Lista nr 6 - KWW KUKIZ\'15']
    #Poniżej trzy komitety, które były zarejestrowane tylko w niektórych okręgach wyborczych
    if 'Lista nr 7 - KWW POLSKA FAIR PLAY BEZPARTYJNI GWIAZDOWSKI' in data_district.columns:
        df['Polska Fair Play'] = data_district['Lista nr 7 - KWW POLSKA FAIR PLAY BEZPARTYJNI GWIAZDOWSKI']
    else:
        df['Polska Fair Play'] = 0
    if 'Lista nr 9 - KKW POLEXIT - KOALICJA' in data_district.columns:
        df['POLEXIT'] = data_district['Lista nr 9 - KKW POLEXIT - KOALICJA']
    else:
        df['POLEXIT'] = 0
    if 'Lista nr 10 - KW JEDNOŚĆ NARODU' in data_district.columns:
        df['Jedność Narodu'] = data_district['Lista nr 10 - KW JEDNOŚĆ NARODU']
    else:
        df['Jedność Narodu'] = 0
    #Dodanie kolumny przechowującej informację o tym, w którym okręgu znajduje się dana komisja
    df['obwod'] = i
    
    df['zwyciezca'] = 0
    df.loc[ df['Prawo i Sprawiedliwość'] > df['Koalicja Europejska'], 'zwyciezca'] = 1
    
    #Doklejenie wczytanych dla okręgu i obrobionych danych do głównego zbioru
    big_dataset = pd.concat([big_dataset,df])

print(big_dataset)


      Kod terytorialny gminy  Nr obwodu głosowania Typ obszaru   
0                     220101                     1        wieś  \
1                     220101                     2        wieś   
2                     220101                     3        wieś   
3                     220102                     1      miasto   
4                     220102                     2      miasto   
...                      ...                   ...         ...   
1836                  326301                    23      miasto   
1837                  326301                    24      miasto   
1838                  326301                    25      miasto   
1839                  326301                    26      miasto   
1840                  326301                    27      miasto   

            Typ obwodu               Siedziba Obwodowej Komisji Wyborczej   
0                stały  Gminna Biblioteka i Ośrodek Kultury w Borzytuc...  \
1                stały            Sala wiejska, Dąbró

In [3]:
features = big_dataset.copy()
features['typ obszaru'] = 0
features.loc[ features['Typ obszaru'] == 'miasto', 'typ'] = 1
features['liczba uprawnionych'] = features['Liczba wyborców uprawnionych do głosowania']
features['liczba glosow'] = features['Liczba kart wyjętych z urny']
features['liczba glosow niewaznych'] = features['Liczba kart wyjętych z urny'] - features['Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów']
features['wojewodztwo'] = 0
wojewodztwa = ['mazowieckie', 'małopolskie', 'wielkopolskie', 'zachodniopomorskie', 'pomorskie', 'warmińsko-mazurskie', 'lubuskie',
              'dolnośląskie', 'śląskie', 'łódzkie', 'opolskie', 'kujawsko-pomorskie', 'podlaskie', 'świętokrzyskie', 'lubelskie',
              'podkarpackie']
for i in range(len(wojewodztwa)):
    features.loc[ features['Województwo'] == wojewodztwa[i], 'wojewodztwo'] = i
    
cols_to_remove = []
for c in range(0,44):
    cols_to_remove.append(c)
#cols_to_remove.append(45)
cols_to_remove.append(47)
#Wycięcie nadmiarowych danych - szczegółowych informacji o głosach na poszczególnych kandydatów w poszczególnych komisjach
features.drop(features.columns[cols_to_remove],axis=1,inplace=True)

print(features)

      obwod  zwyciezca  typ obszaru  liczba uprawnionych  liczba glosow   
0         1          1            0                 1583            549  \
1         1          0            0                  463            154   
2         1          0            0                  424            153   
3         1          1            0                 1550            784   
4         1          0            0                 1392            599   
...     ...        ...          ...                  ...            ...   
1836     13          0            0                 1043            382   
1837     13          0            0                  583            209   
1838     13          0            0                  981            479   
1839     13          0            0                   69             27   
1840     13          0            0                   45             13   

      liczba glosow niewaznych  wojewodztwo  
0                            9            4  
1      

In [4]:
#Wyodrębnienie z danych zbioru testowego
from sklearn.model_selection import train_test_split
data_train, data_test = sklearn.model_selection.train_test_split(features, test_size=2700, random_state=1)

In [5]:
#Wyodrębnienie zbioru walidacyjnego z pozostałego zbioru treningowego
data_train, data_val = sklearn.model_selection.train_test_split(data_train, test_size=2700, random_state=1)

In [6]:
data_test.to_csv('data_test.csv', index = False)

In [8]:
train_labels = np.array(data_train['zwyciezca'])
val_labels = np.array(data_test['zwyciezca'])

data_train.drop(['zwyciezca'], axis=1, inplace=True)
data_val.drop(['zwyciezca'], axis=1, inplace=True)

print(data_val)

      obwod  typ obszaru  liczba uprawnionych  liczba glosow   
1282     10            0                 1527            754  \
616       3            0                  335            116   
1430      7            0                  819            284   
1313      6            0                 1529            704   
1711      3            0                  282             74   
...     ...          ...                  ...            ...   
708       7            0                 1732            526   
115       9            0                  531            241   
2600     12            0                 1280            534   
1962     10            0                 1468            925   
373      13            0                  878            290   

      liczba glosow niewaznych  wojewodztwo  
1282                        10            1  
616                          0           12  
1430                         1            2  
1313                         6            9  
1

In [20]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, activation='relu', input_shape=[len(data_train.keys())]),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'), 
  tf.keras.layers.Dense(1)
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss='mse',
              metrics=['mae', 'mse'])

#model.compile(optimizer='adam',
              #loss='binary_crossentropy',
              #metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 128)               896       
                                                                 
 dense_12 (Dense)            (None, 64)                8256      
                                                                 
 dense_13 (Dense)            (None, 64)                4160      
                                                                 
 dense_14 (Dense)            (None, 1)                 65        
                                                                 
Total params: 13,377
Trainable params: 13,377
Non-trainable params: 0
_________________________________________________________________


In [21]:
history = model.fit(tf.convert_to_tensor(data_train), tf.convert_to_tensor(train_labels), epochs=50, batch_size = 32,
                    validation_data=(tf.convert_to_tensor(data_val), tf.convert_to_tensor(val_labels)))
                    #callbacks=[tensorboard_callback])

train_loss = history.history['loss']
val_loss = history.history['val_loss']
#train_acc = history.history['accuracy']
#val_acc = history.history['val_accuracy']
#test_loss, test_acc = model.evaluate(ds_images_labels_test.batch(32))

#print(f"Test loss: {test_loss}")
#print(f"Test accuracy: {test_acc}")

Epoch 1/50
684/684 [==============================] - 2s 2ms/step - loss: 83.1521 - mae: 4.2273 - mse: 83.1521 - val_loss: 0.3515 - val_mae: 0.5422 - val_mse: 0.3515
Epoch 2/50
684/684 [==============================] - 1s 2ms/step - loss: 0.6305 - mae: 0.5331 - mse: 0.6305 - val_loss: 0.2346 - val_mae: 0.4673 - val_mse: 0.2346
Epoch 3/50
684/684 [==============================] - 1s 2ms/step - loss: 0.2661 - mae: 0.4411 - mse: 0.2661 - val_loss: 0.2356 - val_mae: 0.4577 - val_mse: 0.2356
Epoch 4/50
684/684 [==============================] - 1s 2ms/step - loss: 0.2241 - mae: 0.4291 - mse: 0.2241 - val_loss: 0.2991 - val_mae: 0.4873 - val_mse: 0.2991
Epoch 5/50
684/684 [==============================] - 1s 2ms/step - loss: 0.2217 - mae: 0.4142 - mse: 0.2217 - val_loss: 0.2346 - val_mae: 0.4623 - val_mse: 0.2346
Epoch 6/50
684/684 [==============================] - 1s 2ms/step - loss: 0.2377 - mae: 0.4502 - mse: 0.2377 - val_loss: 0.2502 - val_mae: 0.4743 - val_mse: 0.2502
Epoch 7/50
684

In [17]:
# Zapisanie modelu do pliku
model.save('model_election.h5')